In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Advertising.csv")
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
from sklearn.linear_model import LinearRegression
model  = LinearRegression()

In [8]:
X = df.drop("sales",axis=1)
y=df["sales"]

In [5]:
model.fit(df.drop("sales",axis=1),df["sales"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
model.coef_, model.intercept_

(array([ 0.04576465,  0.18853002, -0.00103749]), 2.9388893694594085)

In [9]:
model.score(X,y)

0.8972106381789521

In [110]:
model_adv.predict([[240,40,70],[240,40,70]])

array([21.39098044, 21.39098044])

In [35]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::618725421363:role/service-role/AmazonSageMaker-ExecutionRole-20211223T020495'

us-east-1


In [138]:
#Build tar file with model data + inference code
bashCommand = "tar -cvpzf model.tar.gz model.joblib inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

sagemaker-us-east-1-618725421363


In [139]:
#Upload tar.gz to bucket
model_artifacts = f"s3://{default_bucket}/model.tar.gz"
response = s3.meta.client.upload_file('model.tar.gz', default_bucket, 'model.tar.gz')

In [140]:
# retrieve sklearn image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="0.23-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)

In [141]:
#Step 1: Model Creation
model_name = "sklearn-test" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: sklearn-test2022-02-08-13-19-13
Model Arn: arn:aws:sagemaker:us-east-1:618725421363:model/sklearn-test2022-02-08-13-19-13


In [142]:
#Step 2: EndPointConfiguration  Creation
sklearn_epc_name = "sklearn-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:618725421363:endpoint-config/sklearn-epc2022-02-08-13-19-14


In [143]:
#Step 3: EndPoint Creation
endpoint_name = "sklearn-local-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:618725421363:endpoint/sklearn-local-ep2022-02-08-13-19-14


In [144]:
#Monitor creation
# %time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'sklearn-local-ep2022-02-08-13-19-14', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:618725421363:endpoint/sklearn-local-ep2022-02-08-13-19-14', 'EndpointConfigName': 'sklearn-epc2022-02-08-13-19-14', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:fc21b1b187c1980fa792c9bd648c34d60e8622ad0963c140a8eb60ceef2fc549', 'ResolutionTime': datetime.datetime(2022, 2, 8, 13, 19, 15, 710000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2022, 2, 8, 13, 19, 14, 970000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(

In [147]:
import boto3
import json
import numpy as np
data =[[240,40,70]]
runtime_client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {"Input": data}
data = json.loads(json.dumps(request_body))
print(data)
payload = json.dumps(data)
print(payload)
endpoint_name = endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)
print(response['Body'].read().decode())
# result = json.loads(response['Body'].read().decode())['Output']
# print(result)

{'Input': [[240, 40, 70]]}
{"Input": [[240, 40, 70]]}
{"Output":21.390980442509672}



In [148]:
client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': 'a04a38c1-4e1a-4922-b623-445e900f79c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a04a38c1-4e1a-4922-b623-445e900f79c4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 08 Feb 2022 13:22:46 GMT'},
  'RetryAttempts': 0}}